# How to call functions with chat models

This notebook demonstrates how to use the Chat Completions API to call external functions to extend the capabilities of GPT models.

In [1]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken --quiet
!pip install termcolor --quiet
!pip install openai --quiet

In [2]:
import json
import random
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-4o"
client = OpenAI()

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Tell a Joke

Let's define a function that tells a random joke.

In [5]:
def get_joke():
    """Returns a random joke."""
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "Why don't some couples go to the gym? Because some relationships don't work out!",
        "I told my wife she should embrace her mistakes. She gave me a hug.",
        "I'm reading a book on anti-gravity. It's impossible to put down!",
    ]
    return random.choice(jokes)

Now let's define the tools we want to use.

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_joke",
            "description": "Get a random joke",
            "parameters": {},
        }
    },
]

Now let's try to get a joke.

In [7]:
messages = []
messages.append({"role": "user", "content": "Tell me a joke"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    if tool_call.function.name == "get_joke":
        joke = get_joke()
        messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": "get_joke", "content": joke})
        final_response = chat_completion_request(messages, tools=tools)
        print(final_response.choices[0].message.content)
